# Designing a Geo-replication Strategy

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000002.png)

Welcome to the module, Designing a Geo‑replication Strategy. Let's see the topics we are going to talk about in this module. We will talk about handling disaster recovery and failover in the popular Azure PaaS services. We will see how to enable multiple Azure regions for an existing Azure Web App and Azure SQL database, and we will also use Azure Front Door service and configure endpoints which support regional failover.

# Failure Mode Analysis for Azure Applications

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000004.png)

Welcome to the clip, Failure Mode Analysis for Azure Applications. Failure mode analysis is the process for building resiliency into a system by identifying possible failure points in the system. We can try to imagine, for instance, that our solution consists of web application, Azure SQL database, to store user data, Azure Cosmos DB, to store JSON documents, and also Azure Storage, to keep some images on the blob storage. Now for each of these components, we should apply failure mode analysis. So we should try to identify possible failure points in these components.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000005.png)

So, what is the general process to conduct a failure mode analysis? We have to identify all of the components in the system. So, for instance, we have to know that we have web application. There is also Web API. There is Azure SQL Database. There is Cosmos DB. For each component, we have to identify potential failures that could occur. For instance, for the web application, it can stop responding. We should also rate each failure mode according to its overall risk. So, we have to answer the question, what will happen and what will be the impact when one of those components in our architecture fails? For each failure mode, we should determine how the application will respond and recover.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000006.png)

When it comes to app service, app shut down, it can be expected shut down, so the application was unloaded because it was idle, but the app could also crash, so it was unexpected shutdown. In this case, we can ask what is the recovery plan and diagnostics?

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000007.png)

If the application was unloaded while idle, it is automatically restarted on the next request. But, to prevent the application from being unloaded while idle, we can enable the Always On setting. If the app crashes or App Service virtual machine becomes unavailable, App Service automatically restarts the application. We should enable diagnostics logging for web apps in Azure App Service to make sure that we can track all the issues and resolve them quickly.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000008.png)

There can be also a broken connection to the SQL database.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000009.png)

Well, in this case, the database must be configured for active geo‑replication. For queries, we should read from a secondary replica. We should also catch two types of exceptions like InvalidOperationException and SqlException in the source code.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000010.png)

There can be also a problem with writing to the storage account.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000011.png)

Then, we should retry the operation to recover from transient failures. The retry policy in the Storage Account SDK handles it automatically. If N retry attempts fail, we should perform a graceful fallback. In this case, we can store the data in a local cache. We should also use storage metrics to discover what happened.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000012.png)

When talking about Azure Cosmos DB data read/write failures, we should catch those two types of exceptions, HttpRequestException and DocumentClientException in the source code. We should also try to replicate the Cosmos DB database across two or more regions. The SDK automatically re tries failed requests. We should also check HttpStatus. When CosmosDB throttles the client, it returns an HTTP 429 error.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000013.png)

When possible, persist the document to a backup queue and process the queue later. Also, log all errors on the client side.

# Web Application in Multiple Azure Regions for High Availability

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000015.png)

Welcome to the clip Web Application in Multiple Azure Regions for High Availability. In this clip, we are going to discuss multiple Azure regions for high availability. This diagram presents Web application deployed in two different Azure regions. Azure SQL Database, Cosmos DB, and Storage Account are also duplicated. In the front of those regions, there is Azure Front Door service. It enables balancing the traffic from the end users on the left side. When one region becomes unavailable, Azure Front Door can route the traffic to the second region.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000016.png)

Let's briefly discuss the architecture. The application is deployed to each region. During normal operations, network traffic is routed to the primary region. If the primary region becomes unavailable, traffic is routed to the secondary region. There is primary and secondary region, there is Azure Front Door service used to load balance the traffic, and there is also geo‑replication enabled for the data stores like Cosmos DB.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000017.png)

If our regional outage affects the primary region, Azure Front Door can be used to failover to the secondary region.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000018.png)

So what are the architectural recommendations? It is recommended to use regional pairing. Choose regions from the same regional pair, like East Europe and West Europe.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000019.png)

Use priority routing. With this setting, Front Door sends all requests to the primary region unless the endpoint for that region becomes unreachable.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000020.png)

Use active geo‑replication for the SQL Database. Failover to the secondary database if your primary database fails or needs to be taken offline.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000021.png)

Use geo‑replication for Azure Cosmos DB. Cosmos DB supports geo‑replication across regions with multi‑master multiple‑write regions.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000022.png)

Use read‑access geo‑redundant Azure storage. With read‑access geo‑redundant storage, the data is replicated to a secondary region. With all these tips, we can be sure that our solution will be more resilient and handling failover will be much easier. Now, it's time for the demo. Let's see how to create Azure Web App in the multiple regions and how to configure Azure Front Door service.

# Demo: Azure Front Door

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000023.png)

In this demo, we will see how to enable multiple Azure regions for the web application and how to create an Azure Front Door service and configure endpoints which support regional failover. Once we sign into the Azure Portal under portal.azure.com we click the Create a resource button. We will start from creating two separate Azure web apps available in different Azure regions. In the Search box we have to type web app and select Web Application. On the next page we click the Create button. There are a few details we have to fill. First of all we have to create a new resource group. To do it we click Create new and provide the name. In my case, this is rg‑pluralsight‑region1. Then we click the OK button. We have to also provide the name of the web app. In this case, psWebAppRegion1. So the app will be available under the psWebAppRegion1.azurewebsites.net URL. Then we have to select the Runtime stack. In this case, let's select .NET Core 2.2. A very important thing, we have to also set a region. In this case, we select East US. Now we have to scroll down and create a new App Service plan. To do it, we have to click Create new, and in the dialogue we have to type the name. In this case, it will be psAppServicePlanEastUsRegion1. Then we click the OK button. We can leave the rest of the settings as they are. So Sku and size is set to Standard S1. Now in the Monitoring tab we can disable Application Insights because we will not use them in this course. The Tags section can be also left empty. In the Review + create view we can see that Region for our web application is set to East US. We click the Create button to create the App Service plan together with web application. To avoid waiting until the resources are provisioned, I forwarded the video. Once the web application is created we can click the Go to resource button. In the Overview tab there are some details about the newly created web app like status, so it's running now; location, so East US; and also there is the URL address. Now we have to create the second App Service plan and web application, but this time in another Azure region, West Europe. I will copy the previous values to modify them during the creation of the second web app. Again, in the main tab we click Create a resource and search for the web app. We click the Create button, and in a displayed blade we have to fill some details like in the previous case. We have to create another resource group. This time it will be called, in my case, rg‑pluralsight‑region2. Once we type the name we click the OK button. Then, we scroll down and provide the web app name, so in this case, psWebAppRegion2. Runtime stack is set to .NET Core 2.2, but this time we set Region to West Europe. We have to also create a separate App Service plan. To do it we click Create new, and we have to type psAppServicePlanWestEuropeRegion2. Once we've typed the name for the App Service plan we click the OK button. The Sku and size section can be left as it is, set to Standard S1. In the Monitoring tab we can disable Application Insights because we will not use this service. We do not have to provide any tags. In the Review + create tab we click the Create button. To avoid waiting until the resources are provisioned, I forwarded the video. Once the web application is created we can click the Go to resource button. This time the application is available in the West Europe region and of course under a different URL. Now it's time to create Azure Front Door service. From the home tab we click the Create a resource button and in the Search box we can type front. From the drop‑down we select Front Door. In the next blade we have to click the Create button. There are a few details we have to fill. First of all we have to select or create our resource group. In this demo, we can use the resource group we created before for the first region, so rg‑pluralsight‑region1. Now, we can go to the Configuration tab. There, we have to provide more details. We click the Plus button on the Frontend hosts, and in the displayed tab we have to provide the host name to route the traffic. Our Front Door service will be available under psfrontend.azurefd.net. The rest of the configuration can stay as it is. We can click the Add button. For the Backend pools, which is step number two, we click the Plus button. Backends are the applications where Front Door service will route the traffic. In our case, we will have two backends, two web applications we created before. From the Backend host type drop‑down, select App service. Let's leave Subscription as it is. From the Backend host name drop‑down we select pswebappregion1.azurewebsites.net. We can leave the rest of the default settings. Then we click the Add button. Now it's time to add a second backend. To do it we click Add a backend. From the Backend host type we select App service. Subscription stays as it is. From the Backend host name drop‑down we select pswebappregion2.azurewebsites.net. We can leave the rest of the default settings and click the Add button. Next, we can scroll down and select the Add button without changing any other settings. Oh, one important thing, we have to provide the name for the new backend pool. In this case, psBackendPool. Now, we click the Plus button on the Routing rules. We have to provide the name. In this case, it will be psRoutingRule. Then, Accepted protocol should be set to HTTP and HTTPS because we would like to load balance the HTTP traffic to our web applications. Now we can scroll down and make sure that the Backend pool is set to the previously created one, so psBackendPool. We can leave all other default settings and click the Add button. Once everything is set as it should be, we can click the Review + create button. After validation we can click the Create button. To avoid waiting until resources are provisioned, I forwarded the video. Once the Front Door service is created, we can access its URL. Let's copy it and paste in the new tab in the browser. Please note that web app will not be displayed immediately after creation. After a few minutes Front Door should be configured. We can try to open the URL once again. This time the welcome page from the Azure web app should be displayed. Now to test the failover we can try to disable the web app in the first region. To do it we have to click the Stop button. Once the web app in the first region is disabled we can try once again to open the URL for the Front Door. We can try to refresh the website a few times. It's still available because of automatic failover to the web app in the second region.

# Summary

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000024.png)

In this module, we discussed failure mode analysis for other applications. We saw how web application is deployed in multiple Azure regions for high availability. We also learned how to create and configure an Azure Front Door service in the Azure portal. This was the last module. I would like to say thank you so much for watching this course. I hope you found it interesting and helpful. Building resilient and high‑available solutions on the Microsoft Azure cloud can be much easier when we know some features that services offer, like active geo‑replication in the Azure SQL Database or multi‑region writes in the Cosmos DB. If you would like to learn more about Azure services we mentioned in this course, I encourage you to check other courses available on Pluralsight. Good luck.

![](https://storage.googleapis.com/agungwahyudi-public-files/azure_disaster_recovery-3-000025.png)